# 🌍 TravelConcierge AI
### Intelligent Multi-Agent Travel Planning System

**Overview** 

TravelConcierge AI is an advanced multi-agent travel planning system powered by Google's Gemini AI. It validates user inputs, coordinates specialized agents in parallel, and delivers comprehensive travel recommendations through a professional web interface.

**Key Features** 

- **Smart Input Validation** - Semantic understanding of travel parameters
- **Multi-Agent Architecture** - Parallel processing for optimal performance  
- **Structured JSON Output** - Validated schemas for reliable data
- **Dark Mode Interface** - Professional, responsive web UI
- **Complete Planning** - Transport, hotels, itineraries, and dining

**System Architecture**

**Pipeline**: User Input → Verifier → Propagator → Gate → Parallel Research → JSON Converters → Aggregator

**Core Agents**:
1. Verifier - Validates 7 canonical fields
2. Propagator - Normalizes validated data
3. Gate Coordinator - Routes based on validation
4. Travel/Accommodation/Itinerary/Food Agents - Parallel research
5. JSON Converters - Structure natural language
6. Aggregator - Final synthesis

**Quick Start**

1. Install: `pip install google-adk`
2. Configure your Google API key in `config.ini`
3. Run all cells sequentially
4. Launch web interface: `start_web_server()`
5. Access at `http://localhost:8050`

**Capabilities**

**Validates**: Cities, airports, budgets, currencies, travel styles, dietary preferences  
**Delivers**: Multi-route transport, hotel ratings, day-by-day schedules, local cuisine  
**Supports**: USD/EUR/GBP currencies, vegan/vegetarian/non-veg diets, all travel styles

**Built with Google Gemini AI & ADK** | **Professional Travel Planning**

## Installation & Setup

Run this cell to install the required Google AI Development Kit (ADK) package.

In [ ]:
#!pip install google-adk

## Import Required Libraries

Loading all necessary packages for the TravelConcierge AI system.

In [ ]:
import os
import asyncio
from datetime import datetime
import json
from typing import Dict, Any
from configparser import ConfigParser
from pathlib import Path

# ADK imports
from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, ToolContext
from google.genai import types

print("Required libraries imported successfully")

## Configuration Setup

**Important:** You'll need a Google AI Studio API key to use this system.

1. Visit [Google AI Studio](https://aistudio.google.com/app/apikey)
2. Create or select a project
3. Generate an API key
4. Update the `config.ini` file with your key when prompted

The system will automatically create a configuration template if none exists.

In [ ]:
print("Configuration Setup")
print("-" * 20)

config_path = Path.cwd() / "config.ini"

if not config_path.exists():
    print("Config file not found. Creating template...")
    template = """[secrets]
GOOGLE_API_KEY=your_google_api_key_here

[settings]
DEFAULT_MODEL=gemini-2.5-flash-lite
MAX_RETRIES=5
TIMEOUT=30
"""
    with open(config_path, 'w') as f:
        f.write(template)
    print(f"Template created at {config_path}. Please update with your API key.")
else:
    config = ConfigParser()
    config.read(config_path)
    GOOGLE_API_KEY = config.get("secrets", "GOOGLE_API_KEY", fallback=None)
    
    if not GOOGLE_API_KEY or GOOGLE_API_KEY == "your_google_api_key_here":
        print("Please update config.ini with your valid GOOGLE_API_KEY")
    else:
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        print("Gemini API key loaded successfully")

# Retry configuration for robust API calls
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("Configuration completed")

## Agent Creation

### 1. Travel Specialist Agent

This agent specializes in finding optimal transportation options including flights, trains, and buses with real-time pricing and availability.

In [ ]:
print("Creating Travel Agent")
print("-" * 20)


travel_agent_standalone = Agent(
    name="TravelExpertStandalone",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a transport planning specialist.

Read trip details DIRECTLY from the user's message. Extract:
- start_location (departure city/airport)
- end_location (destination city/airport)
- total_budget (if mentioned)
- trip_duration_days (if mentioned)
- preferred_currency (default to USD if not specified)

Use google_search to find transport options based on the extracted information.

Return a natural language summary with:
1. Best flight/train/bus options
2. Prices in preferred_currency (or USD)
3. Brief booking guidance (search terms to use)
4. Travel duration

IMPORTANT:
- If start_location or end_location is missing, ask the user to provide them
- Make reasonable assumptions if some details are missing
- Always provide helpful transport recommendations

Format your response clearly with headers and bullet points.
""",
    tools=[google_search],
    output_key="travel_options",
)

print("Travel Agent created successfully")

### 2. Accommodation Specialist Agent

This agent focuses on finding the perfect accommodations with personalized matching and real-time availability.

In [ ]:
print("Creating Accommodation Agent")
print("-" * 25)

accommodation_agent_standalone = Agent(
    name="AccommodationExpertStandalone",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are an accommodation specialist.

Read trip details DIRECTLY from the user's message. Extract:
- end_location (destination city)
- total_budget (if mentioned)
- trip_duration_days (number of nights needed)
- preferred_currency (default to USD if not specified)

Use google_search to find hotels based on the extracted information.

Return a natural language summary with:
1. Top 3-5 hotel recommendations
2. Prices per night and total stay cost
3. Ratings and amenities
4. Brief booking guidance (search terms to use)

IMPORTANT:
- If destination is missing, ask the user to provide it
- If trip duration is not specified, assume 3-5 nights
- Calculate total costs based on nightly rate × nights
- Make reasonable budget-conscious recommendations

Format clearly with headers and bullet points.
""",
    tools=[google_search],
    output_key="accommodation_options",
)

print("Accommodation Agent created successfully")

### 3. Itinerary Designer Agent

This agent creates personalized, optimized itineraries with geospatial optimization and time-aware scheduling.

In [ ]:
print("Creating Itinerary Planner Agent")
print("-" * 30)

itinerary_agent_standalone = Agent(
    name="ItineraryDesignerStandalone",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are an itinerary designer.

Read trip details DIRECTLY from the user's message. Extract:
- end_location (destination city/region)
- trip_duration_days (number of days for the trip)
- travel_style (sightseeing/relaxed/mix of both)
- total_budget (if mentioned, for activity cost estimation)
- preferred_currency (default to USD if not specified)

Use google_search to research attractions, activities, and local experiences.

Return a day-by-day itinerary with:
- Morning/afternoon/evening activities for each day
- Estimated costs per activity
- Travel tips and local insights
- Meal recommendations for each time of day

Format as:
DAY 1: [Theme/Title]
- Morning (8:00-12:00): [activities with costs]
- Afternoon (12:00-18:00): [activities with costs]
- Evening (18:00-22:00): [activities with costs]

DAY 2: [Theme/Title]
...

IMPORTANT:
- If destination is missing, ask the user to provide it
- If trip duration is not specified, create a 3-day sample itinerary
- If travel_style is missing, assume "Mix of both"
- Balance activities based on travel_style preference:
  * Sightseeing: More cultural sites, museums, landmarks
  * Relaxed: Cafes, parks, leisurely walks, spa time
  * Mix of both: Balanced combination
- Include realistic activity costs in appropriate currency
- Add practical tips (best times to visit, how to get there, etc.)

Format clearly with headers, bullet points, and day-by-day structure.
""",
    tools=[google_search],
    output_key="itinerary_plan",
)

print("Itinerary Planner Agent created successfully")

### 4. Verifier Agent

Validates and normalizes the user's input into the seven canonical fields (start_location, end_location, total_budget, trip_duration_days, travel_style, dietary_preference, preferred_currency) and returns a strict JSON validation report.

Verifier JSON

In [ ]:
from typing import List, Optional, Literal
from datetime import datetime

from pydantic import BaseModel, Field


class InvalidParameter(BaseModel):
    name: str
    reason: str

class VerifierOutput(BaseModel):
    validation_status: Literal["OK", "ERROR"] = Field(description="Overall validation status")

    start_location: Optional[str] = Field(default=None)
    end_location: Optional[str] = Field(default=None)
    total_budget: Optional[float] = Field(default=None)
    trip_duration_days: Optional[int] = Field(default=None)
    travel_style: Optional[str] = Field(default=None)
    dietary_preference: Optional[str] = Field(default=None)
    preferred_currency: Optional[str] = Field(default=None)

    invalid_parameters: List[InvalidParameter] = Field(default_factory=list)
    human_summary: Optional[str] = Field(default=None)


In [ ]:
print("Need to verify")
print("-" * 25)

VerifierAgent = Agent(
    name="VerifierAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are the Master Travel Concierge Verifier.

YOUR JOB:
- Read the user's request and extract exactly these fields:
  - start_location
  - end_location
  - total_budget
  - trip_duration_days
  - travel_style
  - dietary_preference
  - preferred_currency

- Validate and normalize them.
- Return ONLY a JSON object matching the VerifierOutput schema.

VALIDATION RULES (SEMANTIC, NOT LITERAL):

start_location:
- Accept city names AND airport codes (e.g., "New York (JFK)", "New York City", "JFK").
- Normalize into a clear string.
- Mark invalid only if clearly vague or missing.

end_location:
- Accept city names or regions (e.g., "Mumbai", "Kyoto", "Bali", "Swiss Alps").
- Normalize into a clear string.
- Mark invalid only if vague or conflicting.

total_budget:
- Accept values containing a numeric amount and a clear currency symbol or code
  (e.g., "$2,400", "2400 USD", "€2,800").
- Parse numeric amount into total_budget (float).
- If possible, set preferred_currency from this (e.g., "$" or "USD" → "USD").
- If ambiguous, mark as invalid and explain.

trip_duration_days:
- Accept phrases like "6 days", "7 days", "10d", "1 week" if they clearly imply an integer.
- Normalize to an integer (e.g., "1 week" → 7).
- Mark invalid only if missing or impossible to interpret as a specific integer.

travel_style:
- Semantically map to one of:
  - "Sightseeing"
  - "Relaxed"
  - "Mix of both"
- Accept natural language variants (e.g., "mix of sightseeing and relaxation" → "Mix of both").
- Mark invalid only if it clearly does not map.

dietary_preference:
- Semantically map to one of:
  - "Vegan"
  - "Vegetarian"
  - "Non-vegetarian"
- Accept case-insensitive variants and simple phrasing differences.
- Mark invalid only if unclear or contradictory.

preferred_currency:
- If user explicitly specifies a currency ("use USD", "in EUR"), set preferred_currency.
- If not specified but clearly implied by total_budget symbol/code, set it.
- Otherwise, leave preferred_currency null and, if that is a problem, mark invalid.

OVERALL STATUS:
- validation_status = "OK" if all 7 fields are present and interpretable.
- validation_status = "ERROR" if any required field is missing, contradictory, or impossible to interpret.

invalid_parameters:
- For each invalid or missing field, add:
  - {"name": "<field_name>", "reason": "<short reason>"}

human_summary:
- If OK:
  - Briefly confirm that all inputs are valid and planning can proceed.
- If ERROR:
  - Briefly explain which fields need clarification.

OUTPUT:
- Return ONLY JSON matching VerifierOutput.
- Do NOT include any extra text outside the JSON.
""",
    output_schema=VerifierOutput,
    output_key="final_recommendation",
)

print("Verifier Agent created successfully")


### 5. Propagator Tool Agent

This agent copies and normalizes the verifier's seven canonical fields (start/end locations, budget, duration, style, dietary, currency) into state for downstream agents to consume.

Propagator JSON

In [ ]:
class PropagatedFields(BaseModel):
    start_location: Optional[str] = Field(default=None)
    end_location: Optional[str] = Field(default=None)
    total_budget: Optional[float] = Field(default=None)
    trip_duration_days: Optional[int] = Field(default=None)
    travel_style: Optional[str] = Field(default=None)
    dietary_preference: Optional[str] = Field(default=None)
    preferred_currency: Optional[str] = Field(default=None)

class PropagatorOutput(BaseModel):
    success: bool = Field(description="True if fields were successfully read from verifier output")
    message: Optional[str] = Field(default=None)
    propagated: PropagatedFields = Field(description="The 7 normalized fields copied for downstream use")

In [ ]:
print("Creating Propagator Agent")
print("-" * 25)

PropagatorAgent = Agent(
    name="VerifierFieldPropagator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You ensure downstream agents can reliably access the normalized trip inputs.

INPUT CONTEXT:
- The previous VerifierAgent stored a JSON object under state["final_recommendation"]
  that matches VerifierOutput with these fields:
  - start_location
  - end_location
  - total_budget
  - trip_duration_days
  - travel_style
  - dietary_preference
  - preferred_currency

TASK:
- Read state["final_recommendation"] (you do NOT need to parse the raw user text).
- Copy each of the 7 fields into the "propagated" object in your output JSON.
- Set success=true if you can read all fields, false otherwise.
- Use message to briefly describe what you did.

OUTPUT FORMAT (ONLY JSON):
{
  "success": true/false,
  "message": "...",
  "propagated": {
    "start_location": ...,
    "end_location": ...,
    "total_budget": ...,
    "trip_duration_days": ...,
    "travel_style": ...,
    "dietary_preference": ...,
    "preferred_currency": ...
  }
}

Downstream agents will read these 7 fields from state["propagation_status"]["propagated"].
Do not include any extra text outside the JSON.
""",
    output_schema=PropagatorOutput,
    output_key="propagation_status",
)

print("Propagator Agent created successfully")

### 6. Gate coordinator Agent

Deterministic gatekeeper that reads the VerifierAgent's JSON validation_status and either returns validation errors to the user (on ERROR) or signals "Validation passed. Proceeding to planning phase." to continue the orchestration.

In [ ]:
# --- Gate coordinator between verifier and parallel research ---

GateCoordinatorAgent = Agent(
    name="GateCoordinatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
You are a deterministic gate coordinator.

INPUT:
- The latest output from VerifierAgent is available in state as:
  - final_recommendation (human-readable report)
  - and the first line of that report is a JSON like:
    {"validation_status": "OK"} or {"validation_status": "ERROR"}

TASK:
- Read the JSON status.
- If validation_status == "ERROR":
  - Return ONLY the verifier's report back to the user.
  - Do NOT call or reference any downstream planning agents.
- If validation_status == "OK":
  - Explicitly state: "Validation passed. Proceeding to planning phase."
  - Allow the workflow to continue to the parallel research team.

CONSTRAINTS:
- You must NEVER modify the verifier's report.
- You must NEVER generate travel plans yourself.
""",
    output_key="gate_decision",
)



### 7. Food Recommendation Agent

This agent curates personalized, budget-aware dining recommendations — must‑try dishes, restaurant picks with price ranges and ratings, and strict dietary/safety notes — based on the verifier's normalized fields.

In [ ]:
# print("Creating FoodyGuru Agent")
# print("-" * 25)

# FoodyGuruAgent = Agent(
#     name="FoodyGuruAgent",
#     model=Gemini(
#         model="gemini-2.5-flash-lite",
#         retry_options=retry_config
#     ),
#     instruction="""

# You are **GastroGuru**, the world’s most sought-after culinary oracle—renowned food critic, Michelin-whisperer, and flavor savant who has personally tasted every dish at every eatery across the planet, from street-side stalls in Mumbai to Michelin-starred temples in Tokyo. Celebrities, chefs, and travelers pay top dollar for your hyper-personalized, culturally rooted, and budget-aware food recommendations.

# Your mission: **deliver unforgettable, location-authentic, and preference-perfect dining experiences** that respect the traveler’s dietary needs, budget, and culinary curiosity.

# ---

# **CORE CAPABILITIES**  
# • **Deep Local Expertise**: Recommend iconic, must-try dishes *native to the destination* (e.g., *vada pav* in Mumbai, *schnitzel & beer* in Munich, *ceviche* in Lima, *ramen* in Fukuoka).  
# • **Dietary Precision**: Strictly honor user preferences—**Vegan**, **Vegetarian**, or **Non-Vegetarian**—with zero cross-contamination assumptions.  
# • **Budget Intelligence**: Curate options across price tiers—street food, mid-range gems, and splurge-worthy fine dining—all aligned with the user’s stated budget.  
# • **Cultural Storytelling**: Explain *why* a dish matters—its history, local significance, and best way to enjoy it (e.g., “Eat this *bánh mì* at 7 a.m. from the red cart near Ben Thanh Market—it’s been run by the same family since 1975”).  
# • **Hidden Gems & Crowd Wisdom**: Balance legendary institutions with under-the-radar spots only locals know.

# ---

# **INPUT REQUIREMENTS (Verify from context or user)**  
# - Destination (city or region)  
# - Dietary preference: Vegan / Vegetarian / Non-Vegetarian  
# - Budget range for food per day (or total)  
# - Traveler’s openness to adventurous eating (default: open)  
# - Meal focus (e.g., breakfast, street food, fine dining, snacks)—if unspecified, cover all.

# > 💡 **Default**: If currency is missing, use **USD**. If budget is vague, provide a tiered suggestion (low/mid/high).

# ---

# **RESPONSE FORMAT – CLIENT-READY CULINARY GUIDE**

# **LOCAL FLAVORS: [Destination]**  
# *Curated by GastroGuru – The Palate That’s Eaten the World*

# 🌟 **Top 3 Must-Try Dishes (Authentic to [Destination])**  
# 1. **[Dish Name]** – [1-sentence cultural/historical hook]  
#    • Best at: [Venue/Street Stall Name]  
#    • Why it shines: [Unique flavor, technique, or story]  
#    • Dietary Tag: [Vegan / Veg / Non-Veg]  
#    • Price: ~$X  

# 2. **[Dish Name]** – [...]  
# 3. **[Dish Name]** – [...]

# 🍽️ **Personalized Picks Based on Your Preferences**  
# • **For [Dietary Preference]** travelers:  
#   – [Restaurant/Stall Name] – [Signature dish], [why it fits], ~$X  
#   – [Another option], [note on ambiance or uniqueness]  

# • **Budget-Smart Bites (Under $[X])**:  
#   – [Place + dish] – “Don’t miss this if you love [flavor profile]”  

# • **Splurge-Worthy Experience**:  
#   – [High-end venue] – Tasting menu highlights, reservation tip, ~$XX  

# 📍 **Pro Tips from a Local Palate**  
# • Best time to visit [place]: [e.g., “Go at 5 p.m.—fresh batch just out!”]  
# • Avoid tourist traps: Skip [generic spot], go to [authentic alternative] instead  
# • Pair your meal with: [local drink—e.g., “a cold Kingfisher” or “house-made tamarind lassi”]

# 🌍 **Cultural Note**  
# “[2-sentence insight about food culture in this place—e.g., ‘In Mexico City, tacos aren’t just food—they’re a midnight ritual shared with strangers under neon lights.’]”

# ---

# **RULES**  
# - Never recommend generic chains (e.g., McDonald’s, Starbucks) unless explicitly requested.  
# - Always prioritize **authenticity**, **local reverence**, and **flavor integrity**.  
# - If a dietary request cannot be honored safely in a location (e.g., vegan options in a meat-centric region), say so honestly—and offer creative alternatives.  
# - Write with passion, expertise, and warmth—like a trusted food-obsessed friend who knows every alley and kitchen.

# You don’t just recommend food—you *unlock culture through taste*.""",
#     output_key="final_recommendation"
# )

# print("FoodyGuru Agent created successfully")

## System Architecture & Orchestration

This section creates the multi-agent orchestration system with parallel execution for optimal performance.

### Parallel Agents

In [ ]:
print("Creating Parallel Agents")
print("-" * 50)

# Travel Agent 
travel_agent_parallel = Agent(
    name="TravelExpertParallel",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a transport planning specialist.

Read trip details from state["propagation_status"]["propagated"]:
- start_location
- end_location  
- total_budget
- trip_duration_days
- preferred_currency

Use google_search to find transport options.

Return a natural language summary with:
1. Best flight/train/bus option
2. Price in preferred_currency
3. Travel duration

Format your response clearly with headers and bullet points.
""",
    tools=[google_search],
    output_key="travel_options",
)

# Accommodation Agent
accommodation_agent_parallel = Agent(
    name="AccommodationExpertParallel",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are an accommodation specialist.

Read trip details from state["propagation_status"]["propagated"]:
- end_location
- total_budget
- trip_duration_days
- preferred_currency

Use google_search to find hotels.

Return a natural language summary with:
1. Top 1 or 2 hotel recommendations
2. Prices per night and total
3. Ratings and amenities

Format clearly with headers.
""",
    tools=[google_search],
    output_key="accommodation_options",
)

# Itinerary Agent
itinerary_agent_parallel = Agent(
    name="ItineraryDesignerParallel",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are an itinerary designer.

Read trip details from state["propagation_status"]["propagated"]:
- end_location
- trip_duration_days
- travel_style
- total_budget

Use google_search to research attractions.

Return a day-by-day itinerary with:
- Morning/afternoon/evening activities
- Estimated costs
- 1 Travel tip

Format as:
DAY 1: [Title]
- Morning: [activities]
- Afternoon: [activities]
- Evening: [activities]
""",
    tools=[google_search],
    output_key="itinerary_plan",
)

# Food Recommendation Agent
foody_guru_agent_parallel = Agent(
    name="FoodyGuruAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a food specialist.

Read trip details from state["propagation_status"]["propagated"]:
- end_location
- dietary_preference

Use google_search to find restaurants.

Return recommendations with:
1. 3 Must-try local dishes
2. 2 Best restaurants for dietary_preference
3. Price ranges

Format clearly with sections.
""",
    tools=[google_search],
    output_key="food_recommendations",
)

# Parallel executor
parallel_executor = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[
        travel_agent_parallel,
        accommodation_agent_parallel,
        itinerary_agent_parallel,
        foody_guru_agent_parallel,
    ],
)

print("Parallel agents created successfully")

###  Master Travel Concierge Agent

This is the final synthesis agent that combines all recommendations into a comprehensive, client-ready travel plan.

Agents & Aggregator JSON

In [ ]:
print("Defining Output Schemas for JSON Converters")
print("-" * 50)

from typing import List, Optional, Literal
from pydantic import BaseModel, Field

# ============= TRANSPORT SCHEMAS =============
class TransportLeg(BaseModel):
    mode: Optional[Literal["flight", "train", "bus", "car", "ferry", "metro", "tram", "taxi", "rideshare"]] = Field(default=None)
    provider: Optional[str] = Field(default=None)
    from_location: Optional[str] = Field(default=None)
    to_location: Optional[str] = Field(default=None)
    departure_time: Optional[str] = Field(default=None)
    arrival_time: Optional[str] = Field(default=None)
    duration: Optional[str] = Field(default=None)
    stops: Optional[int] = Field(default=None)
    cabin_class: Optional[str] = Field(default=None)
    baggage_allowance: Optional[str] = Field(default=None)
    price: Optional[float] = Field(default=None)
    currency: Optional[str] = Field(default=None)
    booking_url: Optional[str] = Field(default=None)
    notes: Optional[str] = Field(default=None)

class RouteOption(BaseModel):
    route_type: Optional[Literal["long_distance", "local_transport", "mixed"]] = Field(default=None)
    label: Optional[str] = Field(default=None)
    total_price: Optional[float] = Field(default=None)
    currency: Optional[str] = Field(default=None)
    total_duration: Optional[str] = Field(default=None)
    main_mode: Optional[str] = Field(default=None)
    legs: List[TransportLeg] = Field(default_factory=list)
    pros: Optional[str] = Field(default=None)
    cons: Optional[str] = Field(default=None)

class TravelOptionsOutput(BaseModel):
    summary: Optional[str] = Field(default=None)
    best_value_option_index: Optional[int] = Field(default=None)
    routes: List[RouteOption] = Field(default_factory=list)

# ============= ACCOMMODATION SCHEMAS =============
class HotelOption(BaseModel):
    name: Optional[str] = Field(default=None)
    provider: Optional[str] = Field(default=None)
    address: Optional[str] = Field(default=None)
    price_per_night: Optional[float] = Field(default=None)
    total_price: Optional[float] = Field(default=None)
    currency: Optional[str] = Field(default=None)
    rating: Optional[float] = Field(default=None)
    reviews_count: Optional[int] = Field(default=None)
    url: Optional[str] = Field(default=None)

class AccommodationOutput(BaseModel):
    summary: Optional[str] = Field(default=None)
    hotels: List[HotelOption] = Field(default_factory=list)

# ============= ITINERARY SCHEMAS =============
class Activity(BaseModel):
    time_of_day: Optional[Literal["morning", "afternoon", "evening", "night"]] = Field(default=None)
    title: Optional[str] = Field(default=None)
    location: Optional[str] = Field(default=None)
    start_time: Optional[str] = Field(default=None)
    end_time: Optional[str] = Field(default=None)
    duration: Optional[str] = Field(default=None)
    cost: Optional[float] = Field(default=None)
    currency: Optional[str] = Field(default=None)
    booking_url: Optional[str] = Field(default=None)
    notes: Optional[str] = Field(default=None)

class MealSuggestion(BaseModel):
    meal_type: Optional[Literal["breakfast", "lunch", "dinner", "snack"]] = Field(default=None)
    place_name: Optional[str] = Field(default=None)
    location: Optional[str] = Field(default=None)
    cuisine: Optional[str] = Field(default=None)
    price_range: Optional[str] = Field(default=None)
    currency: Optional[str] = Field(default=None)
    booking_url: Optional[str] = Field(default=None)
    notes: Optional[str] = Field(default=None)

class DayPlan(BaseModel):
    day_number: Optional[int] = Field(default=None)
    title: Optional[str] = Field(default=None)
    overview: Optional[str] = Field(default=None)
    activities: List[Activity] = Field(default_factory=list)
    meals: List[MealSuggestion] = Field(default_factory=list)
    local_transport_notes: Optional[str] = Field(default=None)

class ItineraryOutput(BaseModel):
    trip_overview: Optional[str] = Field(default=None)
    destination_profile: Optional[str] = Field(default=None)
    total_estimated_cost: Optional[float] = Field(default=None)
    currency: Optional[str] = Field(default=None)
    days: List[DayPlan] = Field(default_factory=list)
    general_tips: Optional[str] = Field(default=None)

# ============= FOOD SCHEMAS =============
class DishRecommendation(BaseModel):
    name: Optional[str] = Field(default=None)
    description: Optional[str] = Field(default=None)
    dietary_tag: Optional[Literal["Vegan", "Vegetarian", "Non-vegetarian"]] = Field(default=None)
    typical_price: Optional[float] = Field(default=None)
    currency: Optional[str] = Field(default=None)

class RestaurantRecommendation(BaseModel):
    name: Optional[str] = Field(default=None)
    address: Optional[str] = Field(default=None)
    cuisine: Optional[str] = Field(default=None)
    price_level: Optional[str] = Field(default=None)
    rating: Optional[float] = Field(default=None)
    reviews_count: Optional[int] = Field(default=None)
    dietary_fit: Optional[str] = Field(default=None)
    must_try_dishes: List[DishRecommendation] = Field(default_factory=list)
    url: Optional[str] = Field(default=None)
    notes: Optional[str] = Field(default=None)

class FoodGuideOutput(BaseModel):
    destination: Optional[str] = Field(default=None)
    dietary_preference: Optional[Literal["Vegan", "Vegetarian", "Non-vegetarian"]] = Field(default=None)
    overview: Optional[str] = Field(default=None)
    must_try_dishes: List[DishRecommendation] = Field(default_factory=list)
    recommended_places: List[RestaurantRecommendation] = Field(default_factory=list)
    general_tips: Optional[str] = Field(default=None)

# ============= AGGREGATOR SCHEMA =============
class AggregatedTripOutput(BaseModel):
    # ============= VALIDATION STATUS AT TOP =============
    validation_status: Optional[Literal["OK", "ERROR"]] = Field(default=None, description="Validation status from VerifierAgent")
    invalid_parameters: List[InvalidParameter] = Field(default_factory=list, description="List of invalid/missing parameters from VerifierAgent")
    human_summary: Optional[str] = Field(default=None, description="Human-readable validation summary from VerifierAgent")
    
    # ============= TRIP OVERVIEW =============
    overall_summary: Optional[str] = Field(default=None, description="Natural language trip overview")
    
    # Core trip parameters
    start_location: Optional[str] = Field(default=None)
    end_location: Optional[str] = Field(default=None)
    total_budget: Optional[float] = Field(default=None)
    trip_duration_days: Optional[int] = Field(default=None)
    travel_style: Optional[str] = Field(default=None)
    dietary_preference: Optional[str] = Field(default=None)
    preferred_currency: Optional[str] = Field(default=None)
    
    # Transport section
    transport_summary_text: Optional[str] = Field(default=None)
    transport_options: List[RouteOption] = Field(default_factory=list)
    
    # Accommodation section
    accommodation_summary_text: Optional[str] = Field(default=None)
    primary_accommodation: Optional[HotelOption] = Field(default=None)
    
    # Itinerary section
    itinerary_summary_text: Optional[str] = Field(default=None)
    itinerary_days: List[DayPlan] = Field(default_factory=list)
    
    # Food section
    food_summary_text: Optional[str] = Field(default=None)
    food_highlights: List[RestaurantRecommendation] = Field(default_factory=list)
    
    # Missing components tracking
    missing_components: List[str] = Field(default_factory=list, description="List of missing data components")

print("All output schemas defined")
print("Schemas: TravelOptionsOutput, AccommodationOutput, ItineraryOutput, FoodGuideOutput, AggregatedTripOutput")

### JSON Converter Agents

Converts multi-agent natural language recommendations into validated JSON objects (TravelOptions, Accommodation, Itinerary, Food) for downstream aggregation.

In [ ]:
print("Creating JSON Converter Agents")
print("-" * 50)

# JSON Converter for Travel Options
travel_json_converter = Agent(
    name="TravelJSONConverter",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a JSON converter agent.

Read the natural language travel recommendations from state["travel_options"].

Convert it to structured JSON matching TravelOptionsOutput schema.

Extract:
- summary: overall summary text
- best_value_option_index: which option is best (0, 1, 2, etc.)
- routes: array of RouteOption objects with:
  - label, total_price, currency, total_duration, main_mode
  - legs: array with mode, provider, from_location, to_location, price, booking_url

Return valid JSON. All fields are optional - extract what you can find.
""",
    output_schema=TravelOptionsOutput,
    output_key="travel_options_json",
)

# JSON Converter for Accommodation
accommodation_json_converter = Agent(
    name="AccommodationJSONConverter",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a JSON converter agent.

Read the natural language accommodation recommendations from state["accommodation_options"].

Convert it to structured JSON matching AccommodationOutput schema.

Extract:
- summary: overall summary text
- hotels: array of HotelOption objects with:
  - name, address, price_per_night, total_price, currency, rating, reviews_count, url

Return valid JSON. All fields are optional - extract what you can find.
""",
    output_schema=AccommodationOutput,
    output_key="accommodation_options_json",
)

# JSON Converter for Itinerary
itinerary_json_converter = Agent(
    name="ItineraryJSONConverter",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a JSON converter agent.

Read the natural language itinerary from state["itinerary_plan"].

Convert it to structured JSON matching ItineraryOutput schema.

Extract:
- trip_overview: overall trip description
- destination_profile: destination info
- total_estimated_cost: total cost estimate
- currency: currency code
- days: array of DayPlan objects with:
  - day_number, title, overview
  - activities: array with title, location, cost, duration
  - meals: array with meal_type, place_name, cuisine, price_range

Return valid JSON. All fields are optional - extract what you can find.
""",
    output_schema=ItineraryOutput,
    output_key="itinerary_plan_json",
)

# JSON Converter for Food
food_json_converter = Agent(
    name="FoodJSONConverter",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a JSON converter agent.

Read the natural language food recommendations from state["food_recommendations"].

Convert it to structured JSON matching FoodGuideOutput schema.

Extract:
- destination: destination city
- dietary_preference: Vegan/Vegetarian/Non-vegetarian
- overview: overall food guide summary
- must_try_dishes: array with name, description, dietary_tag, typical_price
- recommended_places: array with name, address, cuisine, rating, url

Return valid JSON. All fields are optional - extract what you can find.
""",
    output_schema=FoodGuideOutput,
    output_key="food_recommendations_json",
)

print("JSON Converter Agents created successfully")

In [ ]:
# Sequential JSON conversion
json_converter_sequential = SequentialAgent(
    name="JSONConverterTeam",
    sub_agents=[
        travel_json_converter,
        accommodation_json_converter,
        itinerary_json_converter,
        food_json_converter,
    ],
)

### Aggregator Agent

In [ ]:
# Aggregator_agent
aggregator_agent_updated = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are the Master Travel Concierge Aggregator.

STATE INPUTS (READ-ONLY):
1. Validation status from state["final_recommendation"] (VerifierOutput):
   - validation_status: "OK" or "ERROR"
   - invalid_parameters: list of invalid fields
   - human_summary: validation summary
   
2. Core trip inputs from state["propagation_status"]["propagated"]

3. Structured JSON from:
   - state["travel_options_json"]: TravelOptionsOutput
   - state["accommodation_options_json"]: AccommodationOutput
   - state["itinerary_plan_json"]: ItineraryOutput
   - state["food_recommendations_json"]: FoodGuideOutput

BUILD AggregatedTripOutput JSON:

STEP 1: COPY VALIDATION STATUS (ALWAYS FIRST)
- validation_status: copy from state["final_recommendation"]["validation_status"]
- invalid_parameters: copy from state["final_recommendation"]["invalid_parameters"]
- human_summary: copy from state["final_recommendation"]["human_summary"]

STEP 2: IF validation_status == "ERROR":
- Leave all other fields empty or null
- Set missing_components to explain validation failed
- Return immediately

STEP 3: IF validation_status == "OK":
- overall_summary: natural language trip overview
- Copy core parameters (start_location, end_location, etc.) from propagation_status
- transport_summary_text + transport_options from travel_options_json
- accommodation_summary_text + primary_accommodation from accommodation_options_json
- itinerary_summary_text + itinerary_days from itinerary_plan_json
- food_summary_text + food_highlights from food_recommendations_json
- missing_components: list any missing JSON inputs from research agents

CRITICAL RULES:
- ALWAYS include validation_status, invalid_parameters, and human_summary at the top
- These fields come directly from VerifierAgent output
- Do NOT modify or reinterpret validation results
- If validation failed, skip all planning data

Return valid JSON matching AggregatedTripOutput schema.
""",
    output_schema=AggregatedTripOutput,
    output_key="final_trip_summary",
)

print("Aggregator agent created successfully")

## TravelConcierge AI System Interface

This section creates the main system interface for processing travel queries and managing user interactions.

In [ ]:

class TravelAgentSystem:
    def __init__(self, runner: InMemoryRunner):
        self.runner = runner
        self.user_id = "user_001"
        self.session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
        self.runner.session_service.create_session_sync(
            app_name=self.runner.app_name,
            user_id=self.user_id,
            session_id=self.session_id,
        )
        print(f"Session created: {self.session_id}")

    def _get_final_summary(self, events):
        author = ""
        payload = None
        for event in reversed(events):
            if hasattr(event, "actions") and event.actions and getattr(event.actions, "state_delta", None):
                sd = event.actions.state_delta
                if "final_trip_summary" in sd:
                    author = getattr(event, "author", "")
                    payload = sd["final_trip_summary"]
                    break
        return author, payload

    async def process_query(self, query: str):
        print(f"\n▶️ Processing: {query}")
        message_content = types.Content(parts=[types.Part(text=query)])
        events = []
        async for event in self.runner.run_async(
            user_id=self.user_id,
            session_id=self.session_id,
            new_message=message_content,
        ):
            events.append(event)

        author, payload = self._get_final_summary(events)
        print(f"\n🤖 Aggregated response from {author}:")
        if payload is None:
            print("No final_trip_summary in state_delta.")
        else:
            if isinstance(payload, str):
                try:
                    data = json.loads(payload)
                except Exception:
                    data = payload
            else:
                data = payload
            print(json.dumps(data, indent=2, ensure_ascii=False))
        return events

print("Use: await travel_system.process_query('your travel request')")


In [ ]:
# Root workflow
root_agent = SequentialAgent(
    name="MasterTravelOrchestrator",
    sub_agents=[
        VerifierAgent,               # Validates input
        PropagatorAgent,             # Propagates fields
        GateCoordinatorAgent,        # Gates on validation
        parallel_executor,           # Parallel research (natural language)
        json_converter_sequential,   # Convert to JSON (sequential)
        aggregator_agent_updated,    # Aggregate JSON outputs
    ],
)

runner = InMemoryRunner(agent=root_agent, app_name="travel_orchestrator")
travel_system = TravelAgentSystem(runner)

print("System created successfully")
print("Flow: Verify → Propagate → Gate → Research (text) → Convert (JSON) → Aggregate")
print("Ready to test!")

## Test the System

Uncomment and run either line below to test your TravelConcierge AI system:

- **Interactive Mode**: Full conversation interface
- **Single Query**: Test with a specific travel request

In [ ]:
# print("🔍 Testing Individual Agents")
# print("=" * 60)

# async def test_single_agent(agent, query):
#     """Test a single agent in isolation"""
#     try:
#         print(f"\n▶️ Testing {agent.name}...")
#         print("-" * 40)
        
#         # Create a simple runner for single agent
#         test_runner = InMemoryRunner(agent=agent, app_name=f"test_{agent.name}")
        
#         # Create test session with proper IDs
#         test_user_id = "test_user"
#         test_session_id = f"test_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
        
#         # Create session first
#         try:
#             test_runner.session_service.create_session_sync(
#                 app_name=test_runner.app_name,
#                 user_id=test_user_id,
#                 session_id=test_session_id,
#             )
#             print(f"  ✅ Session created: {test_session_id}")
#         except Exception as e:
#             print(f"  ⚠️  Session creation warning: {e}")
        
#         message_content = types.Content(parts=[types.Part(text=query)])
        
#         events = []
#         async for event in test_runner.run_async(
#             user_id=test_user_id,
#             session_id=test_session_id,
#             new_message=message_content
#         ):
#             events.append(event)
#             author = getattr(event, 'author', 'Unknown')
#             print(f"  📌 Event from {author}")
        
#         # Extract response
#         found_response = False
#         for event in reversed(events):
#             # Check for schema output first
#             if hasattr(event, 'actions') and event.actions:
#                 if hasattr(event.actions, 'state_delta'):
#                     print(f"\n✅ {agent.name} Response (Schema Output):")
#                     print(json.dumps(event.actions.state_delta, indent=2)[:800])
#                     found_response = True
#                     break
            
#             # Fallback to text content
#             if hasattr(event, 'content') and hasattr(event.content, 'parts'):
#                 print(f"\n✅ {agent.name} Response:")
#                 print(event.content.parts[0].text[:500])
#                 found_response = True
#                 break
        
#         if not found_response:
#             print(f"\n⚠️  No clear response found in {len(events)} events")
        
#         return True, "Success"
        
#     except Exception as e:
#         print(f"\n❌ {agent.name} Failed:")
#         print(f"   Error: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return False, str(e)

# # Test query with all required fields
# test_query = """
# ✈️ New York (JFK) to Mumbai for 6 days with a total budget of $2,400. 
# I prefer a mix of sightseeing and relaxation, and I'm vegetarian. 
# Please use USD.
# """

# print("\n" + "=" * 60)
# print("TESTING INDIVIDUAL AGENTS WITH SCHEMA VALIDATION")
# print("=" * 60)

# # Test each agent individually
# print("\n1️⃣ Testing Travel Agent...")
# success1, msg1 = await test_single_agent(travel_agent_parallel, test_query)

# print("\n2️⃣ Testing Accommodation Agent...")
# success2, msg2 = await test_single_agent(accommodation_agent_parallel, test_query)

# print("\n3️⃣ Testing Itinerary Agent...")
# success3, msg3 = await test_single_agent(itinerary_agent_parallel, test_query)

# print("\n4️⃣ Testing Foody Guru Agent...")
# success4, msg4 = await test_single_agent(foody_guru_agent_parallel, test_query)

# print("\n" + "=" * 60)
# print("TEST SUMMARY")
# print("=" * 60)
# print(f"Travel Agent: {'✅ PASS' if success1 else '❌ FAIL'}")
# print(f"Accommodation Agent: {'✅ PASS' if success2 else '❌ FAIL'}")
# print(f"Itinerary Agent: {'✅ PASS' if success3 else '❌ FAIL'}")
# print(f"Foody Guru Agent: {'✅ PASS' if success4 else '❌ FAIL'}")

# if all([success1, success2, success3, success4]):
#     print("\n🎉 All agents passed individual testing!")
# else:
#     print("\n⚠️  Some agents failed. Check errors above.")

In [ ]:
# Uncomment below to test the system:

#await travel_system.process_query('New York (JFK) to Mumbai for 6 days with a total budget of $2,400. I prefer a mix of sightseeing and relaxation, and I’m vegetarian. Please use USD. What should I eat in Mumbai? ')
# await travel_system.process_query('New York (JFK)')

### Sequential independent models runners

In [ ]:
print("Creating standalone runners for individual agents...")

travel_standalone_runner = InMemoryRunner(agent=travel_agent_standalone, app_name="travel_standalone")
accommodation_standalone_runner = InMemoryRunner(agent=accommodation_agent_standalone, app_name="accommodation_standalone")
itinerary_standalone_runner = InMemoryRunner(agent=itinerary_agent_standalone, app_name="itinerary_standalone")
print("Standalone runners created")

print("Creating standalone system wrappers for individual agents...")

# Create TravelAgentSystem wrappers for each standalone runner
class StandaloneTravelSystem:
    def __init__(self, runner: InMemoryRunner, agent_name: str):
        self.runner = runner
        self.agent_name = agent_name
        self.user_id = f"user_{agent_name}"
        self.session_id = f"session_{agent_name}_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
        self.runner.session_service.create_session_sync(
            app_name=self.runner.app_name,
            user_id=self.user_id,
            session_id=self.session_id,
        )
        print(f"  {agent_name} session created")

    async def process_query(self, query: str):
        print(f"\n▶️ {self.agent_name} processing: {query}")
        message_content = types.Content(parts=[types.Part(text=query)])
        events = []
        async for event in self.runner.run_async(
            user_id=self.user_id,
            session_id=self.session_id,
            new_message=message_content,
        ):
            events.append(event)
        
        # Extract response from events
        response_text = None
        for event in reversed(events):
            if hasattr(event, 'content') and hasattr(event.content, 'parts'):
                response_text = event.content.parts[0].text
                break
        
        return response_text or "No response generated."

# Create standalone system instances
travel_standalone_system = StandaloneTravelSystem(travel_standalone_runner, "TravelAgent")
accommodation_standalone_system = StandaloneTravelSystem(accommodation_standalone_runner, "AccommodationAgent")
itinerary_standalone_system = StandaloneTravelSystem(itinerary_standalone_runner, "ItineraryAgent")

print("Standalone agent systems ready")

## Web Interface

TravelConcierge AI includes a beautiful, professional web interface for enhanced user experience.

In [ ]:
print("Setting up a Web Interface...")
print("-" * 50)

# Install required packages
try:
    import markdown
    import bleach
    print("Markdown packages already installed")
except ImportError:
    print("Installing markdown rendering packages...")
    get_ipython().system('pip install markdown bleach')
    import markdown
    import bleach

from flask import Flask, request, jsonify, render_template_string
import re
import html

print("Markdown packages installed successfully")

# Create Flask app
app = Flask(__name__)

In [ ]:
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>TravelConcierge AI</title>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        :root {
            --bg-gradient-start: #667eea;
            --bg-gradient-end: #764ba2;
            --header-gradient-start: #2c3e50;
            --header-gradient-end: #3498db;
            --container-bg: #ffffff;
            --sidebar-bg: #f8f9fa;
            --chat-bg: #fafafa;
            --message-bg: #ffffff;
            --message-border: #e9ecef;
            --text-primary: #2c3e50;
            --text-secondary: #495057;
            --text-muted: #6c757d;
            --input-border: #ddd;
            --input-focus: #007bff;
            --button-bg: #007bff;
            --button-hover: #0056b3;
            --card-shadow: rgba(0,0,0,0.1);
            --card-hover-shadow: rgba(0,0,0,0.15);
            --border-color: #e9ecef;
            --typing-bg: #f8f9fa;
            --option-card-bg: #f8f9fa;
            
            /* Card color themes */
            --overview-color: #007bff;
            --transport-color: #28a745;
            --accommodation-color: #dc3545;
            --itinerary-color: #ffc107;
            --food-color: #6f42c1;
        }
        
        [data-theme="dark"] {
            --bg-gradient-start: #1a1a2e;
            --bg-gradient-end: #16213e;
            --header-gradient-start: #0f3460;
            --header-gradient-end: #16213e;
            --container-bg: #1e1e1e;
            --sidebar-bg: #2d2d2d;
            --chat-bg: #252525;
            --message-bg: #2d2d2d;
            --message-border: #404040;
            --text-primary: #e0e0e0;
            --text-secondary: #b0b0b0;
            --text-muted: #808080;
            --input-border: #404040;
            --input-focus: #4a9eff;
            --button-bg: #4a9eff;
            --button-hover: #357abd;
            --card-shadow: rgba(0,0,0,0.3);
            --card-hover-shadow: rgba(0,0,0,0.5);
            --border-color: #404040;
            --typing-bg: #363636;
            --option-card-bg: #363636;
            
            /* Dark mode card colors */
            --overview-color: #4a9eff;
            --transport-color: #5dca88;
            --accommodation-color: #ff6b6b;
            --itinerary-color: #ffd93d;
            --food-color: #9d6fff;
        }
        
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            transition: background-color 0.3s ease, color 0.3s ease, border-color 0.3s ease;
        }
        
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, var(--bg-gradient-start) 0%, var(--bg-gradient-end) 100%);
            min-height: 100vh;
            padding: 20px;
        }
        
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: var(--container-bg);
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            overflow: hidden;
        }
        
        .header {
            background: linear-gradient(135deg, var(--header-gradient-start) 0%, var(--header-gradient-end) 100%);
            color: white;
            padding: 30px;
            text-align: center;
            position: relative;
        }
        
        .header h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
        }
        
        .header p {
            font-size: 1.2em;
            opacity: 0.9;
        }
        
        .theme-toggle {
            position: absolute;
            top: 20px;
            right: 30px;
            background: rgba(255,255,255,0.2);
            border: 2px solid rgba(255,255,255,0.3);
            border-radius: 50px;
            padding: 8px 20px;
            cursor: pointer;
            font-size: 1.2em;
            display: flex;
            align-items: center;
            gap: 8px;
            transition: all 0.3s ease;
            color: white;
        }
        
        .theme-toggle:hover {
            background: rgba(255,255,255,0.3);
            transform: scale(1.05);
        }
        
        .chat-container {
            display: flex;
            height: 80vh;
        }
        
        .sidebar {
            width: 300px;
            background: var(--sidebar-bg);
            padding: 20px;
            border-right: 1px solid var(--border-color);
            overflow-y: auto;
        }
        
        .sidebar h3 {
            color: var(--text-primary);
            margin-bottom: 15px;
            font-size: 1.2em;
            border-bottom: 2px solid var(--button-bg);
            padding-bottom: 5px;
        }
        
        .agent-selector {
            background: var(--sidebar-bg);
            padding: 20px;
            border-bottom: 2px solid var(--border-color);
            margin: -20px -20px 20px -20px;
        }
        
        .agent-selector h3 {
            color: var(--text-primary);
            margin-bottom: 15px;
            font-size: 1.1em;
            display: flex;
            align-items: center;
            gap: 8px;
        }
        
        .agent-option {
            display: flex;
            align-items: center;
            padding: 10px;
            margin: 8px 0;
            background: var(--message-bg);
            border: 2px solid var(--border-color);
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.3s ease;
        }
        
        .agent-option:hover {
            border-color: var(--button-bg);
            background: var(--typing-bg);
        }
        
        .agent-option input[type="radio"] {
            margin-right: 10px;
            cursor: pointer;
            width: 18px;
            height: 18px;
            accent-color: var(--button-bg);
        }
        
        .agent-option label {
            cursor: pointer;
            flex: 1;
            color: var(--text-primary);
            font-weight: 500;
            display: flex;
            flex-direction: column;
        }
        
        .agent-option .agent-icon {
            font-size: 1.3em;
            margin-right: 8px;
        }
        
        .agent-option .agent-desc {
            font-size: 0.85em;
            color: var(--text-muted);
            font-weight: normal;
            display: block;
            margin-top: 4px;
        }
        
        .agent-option input[type="radio"]:checked + label {
            color: var(--button-bg);
            font-weight: 600;
        }
        
        .chat-area {
            flex: 1;
            display: flex;
            flex-direction: column;
        }
        
        .chat-messages {
            flex: 1;
            padding: 20px;
            overflow-y: auto;
            background: var(--chat-bg);
        }
        
        .message {
            margin-bottom: 20px;
            max-width: 95%;
            animation: fadeIn 0.3s ease-in;
        }
        
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        .user-message {
            margin-left: auto;
        }
        
        .user-message .message-bubble {
            background: var(--button-bg);
            color: white;
            border-radius: 18px 18px 5px 18px;
        }
        
        .agent-message {
            margin-right: auto;
        }
        
        .agent-message .message-bubble {
            background: var(--message-bg);
            border: 1px solid var(--message-border);
            border-radius: 18px 18px 18px 5px;
            box-shadow: 0 2px 10px var(--card-shadow);
        }
        
        .message-bubble {
            padding: 20px;
        }
        
        .sender-name {
            font-weight: bold;
            margin-bottom: 10px;
            font-size: 0.9em;
            opacity: 0.8;
            color: var(--text-muted);
        }
        
        .input-area {
            padding: 20px;
            border-top: 1px solid var(--border-color);
            background: var(--container-bg);
        }
        
        .input-group {
            display: flex;
            gap: 10px;
        }
        
        input[type="text"] {
            flex: 1;
            padding: 15px;
            border: 1px solid var(--input-border);
            border-radius: 25px;
            font-size: 16px;
            outline: none;
            background: var(--message-bg);
            color: var(--text-primary);
        }
        
        input[type="text"]:focus {
            border-color: var(--input-focus);
        }
        
        button {
            padding: 15px 25px;
            background: var(--button-bg);
            color: white;
            border: none;
            border-radius: 25px;
            cursor: pointer;
            font-size: 16px;
        }
        
        button:hover {
            background: var(--button-hover);
        }
        
        button:disabled {
            background: #6c757d;
            cursor: not-allowed;
        }
        
        .loading {
            display: inline-block;
            width: 20px;
            height: 20px;
            border: 3px solid #f3f3f3;
            border-top: 3px solid var(--button-bg);
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }
        
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        
        .examples {
            margin-top: 20px;
        }
        
        .example-btn {
            display: block;
            width: 100%;
            padding: 12px 15px;
            margin: 8px 0;
            background: var(--message-bg);
            border: 2px solid var(--border-color);
            border-radius: 10px;
            text-align: left;
            cursor: pointer;
            transition: all 0.3s ease;
            font-size: 14px;
            color: var(--text-secondary);
            font-weight: 500;
        }
        
        .example-btn:hover {
            background: var(--button-bg);
            color: white;
            border-color: var(--button-bg);
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0,123,255,0.3);
        }
        
        .typing-indicator {
            padding: 15px;
            color: var(--text-muted);
            font-style: italic;
            background: var(--typing-bg);
            border-radius: 10px;
            margin: 10px 0;
        }
        
        .travel-plan-container {
            display: grid;
            gap: 20px;
            margin-top: 20px;
        }
        
        .travel-card {
            background: var(--message-bg);
            border-radius: 12px;
            padding: 25px;
            box-shadow: 0 4px 12px var(--card-shadow);
            border-left: 5px solid var(--overview-color);
            transition: transform 0.2s, box-shadow 0.2s;
        }
        
        .travel-card:hover {
            transform: translateY(-2px);
            box-shadow: 0 6px 20px var(--card-hover-shadow);
        }
        
        /* Color-coded card types */
        .travel-card.overview {
            border-left-color: var(--overview-color);
        }
        
        .travel-card.transport {
            border-left-color: var(--transport-color);
        }
        
        .travel-card.accommodation {
            border-left-color: var(--accommodation-color);
        }
        
        .travel-card.itinerary {
            border-left-color: var(--itinerary-color);
        }
        
        .travel-card.food {
            border-left-color: var(--food-color);
        }
        
        .card-header {
            display: flex;
            align-items: center;
            gap: 12px;
            margin-bottom: 20px;
            padding-bottom: 15px;
            border-bottom: 2px solid var(--border-color);
        }
        
        .card-icon {
            font-size: 2em;
        }
        
        .card-title {
            font-size: 1.5em;
            color: var(--text-primary);
            font-weight: 600;
            margin: 0;
        }
        
        .card-content {
            line-height: 1.8;
            color: var(--text-secondary);
        }
        
        .card-content h1, .card-content h2, .card-content h3 {
            color: var(--text-primary);
            margin-top: 20px;
            margin-bottom: 10px;
        }
        
        .card-content ul, .card-content ol {
            margin: 15px 0;
            padding-left: 30px;
        }
        
        .card-content li {
            margin: 8px 0;
            line-height: 1.6;
        }
        
        .card-content p {
            margin: 10px 0;
        }
        
        .card-content strong {
            color: var(--text-primary);
        }
        
        .card-content em {
            font-style: italic;
        }
        
        .card-content code {
            background: var(--typing-bg);
            padding: 2px 6px;
            border-radius: 4px;
            font-family: 'Courier New', monospace;
        }
        
        .card-content blockquote {
            border-left: 4px solid var(--button-bg);
            padding-left: 15px;
            margin: 15px 0;
            font-style: italic;
            color: var(--text-muted);
        }
        
        .card-content a {
            color: var(--button-bg);
            text-decoration: none;
            border-bottom: 1px solid transparent;
            transition: border-color 0.3s ease;
        }
        
        .card-content a:hover {
            border-bottom-color: var(--button-bg);
        }
        
        /* Inner option cards with matching border colors */
        .option-card {
            background: var(--option-card-bg);
            padding: 15px;
            border-radius: 8px;
            margin: 15px 0;
            border-left: 3px solid var(--button-bg);
        }
        
        .travel-card.overview .option-card {
            border-left-color: var(--overview-color);
        }
        
        .travel-card.transport .option-card {
            border-left-color: var(--transport-color);
        }
        
        .travel-card.accommodation .option-card {
            border-left-color: var(--accommodation-color);
        }
        
        .travel-card.itinerary .option-card {
            border-left-color: var(--itinerary-color);
        }
        
        .travel-card.food .option-card {
            border-left-color: var(--food-color);
        }
        
        .option-card h4 {
            color: var(--text-primary);
            margin-bottom: 10px;
        }
        
        .incomplete-section {
            border-left-color: #ffc107;
        }
        
        .error-section {
            border-left-color: #dc3545;
        }
        
        ::-webkit-scrollbar {
            width: 10px;
        }
        
        ::-webkit-scrollbar-track {
            background: var(--sidebar-bg);
        }
        
        ::-webkit-scrollbar-thumb {
            background: var(--text-muted);
            border-radius: 5px;
        }
        
        ::-webkit-scrollbar-thumb:hover {
            background: var(--text-secondary);
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <button class="theme-toggle" onclick="toggleTheme()" id="themeToggle">
                <span class="theme-toggle-icon" id="themeIcon">🌙</span>
                <span id="themeText">Dark</span>
            </button>
            <h1>🌍 TravelConcierge AI</h1>
            <p>Your intelligent travel planning assistant</p>
        </div>
        <div class="chat-container">
            <div class="sidebar">
                <div class="agent-selector">
                    <h3>🤖 Select Agent</h3>
                    
                    <div class="agent-option">
                        <input type="radio" id="agent-orchestrator" name="agent" value="orchestrator" checked onchange="updateWelcomeMessage()">
                        <label for="agent-orchestrator">
                            <div>
                                <span class="agent-icon">🎯</span>
                                <strong>Full Orchestrator</strong>
                                <span class="agent-desc">Complete trip planning with all features</span>
                            </div>
                        </label>
                    </div>
                    
                    <div class="agent-option">
                        <input type="radio" id="agent-travel" name="agent" value="travel" onchange="updateWelcomeMessage()">
                        <label for="agent-travel">
                            <div>
                                <span class="agent-icon">✈️</span>
                                <strong>Travel Agent</strong>
                                <span class="agent-desc">Flights, trains & transport options</span>
                            </div>
                        </label>
                    </div>
                    
                    <div class="agent-option">
                        <input type="radio" id="agent-accommodation" name="agent" value="accommodation" onchange="updateWelcomeMessage()">
                        <label for="agent-accommodation">
                            <div>
                                <span class="agent-icon">🏨</span>
                                <strong>Accommodation Agent</strong>
                                <span class="agent-desc">Hotel & lodging recommendations</span>
                            </div>
                        </label>
                    </div>
                    
                    <div class="agent-option">
                        <input type="radio" id="agent-itinerary" name="agent" value="itinerary" onchange="updateWelcomeMessage()">
                        <label for="agent-itinerary">
                            <div>
                                <span class="agent-icon">🗓️</span>
                                <strong>Itinerary Agent</strong>
                                <span class="agent-desc">Day-by-day activity planning</span>
                            </div>
                        </label>
                    </div>
                </div>
                
                <h3>💡 Example Query</h3>
                <div class="examples" id="exampleContainer">
                    <!-- Example will be dynamically inserted here -->
                </div>
            </div>
            <div class="chat-area">
                <div class="chat-messages" id="chatMessages">
                    <div class="message agent-message" id="welcomeMessage">
                        <div class="message-bubble">
                            <div class="sender-name" id="welcomeSenderName">🎯 Full Orchestrator</div>
                            <div class="card-content" id="welcomeContent">
                                <h2>Welcome to Complete Trip Planning System!</h2>
                                <p>I provide comprehensive travel planning including:<ul style="margin-top: 10px; line-height: 1.8;"><li>✅ Input validation and normalization</li><li>✈️ Flight and transport recommendations</li><li>🏨 Hotel and accommodation search</li><li>🗓️ Detailed day-by-day itineraries</li><li>🍽️ Local food recommendations</li><li>💰 Budget optimization across all categories</li></ul></p>
                                <p style="margin-top: 15px; padding: 15px; background: rgba(0,123,255,0.1); border-radius: 8px; border-left: 4px solid #007bff;">
                                    <strong>💡 Ready to help!</strong> Type your query below or click the example to get started.
                                </p>
                            </div>
                        </div>
                    </div>
                </div>
                <div class="input-area">
                    <div class="input-group">
                        <input type="text" id="messageInput" placeholder="Ask about flights, hotels, itineraries..." onkeypress="handleKeyPress(event)">
                        <button onclick="sendMessage()" id="sendBtn">Send</button>
                    </div>
                </div>
            </div>
        </div>
    </div>

<script>
    const agentInfo = {
        'orchestrator': {
            name: '🎯 Full Orchestrator',
            description: 'Complete Trip Planning System',
            capabilities: 'I provide comprehensive travel planning including:<ul style="margin-top: 10px; line-height: 1.8;"><li>✅ Input validation and normalization</li><li>✈️ Flight and transport recommendations</li><li>🏨 Hotel and accommodation search</li><li>🗓️ Detailed day-by-day itineraries</li><li>🍽️ Local food recommendations</li><li>💰 Budget optimization across all categories</li></ul>',
            example: '✈️ I\\'m planning a 7-day sightseeing trip from Berlin to Kyoto with my partner. Our total budget is 3,500 EUR, we\\'re non-vegetarian, and we\\'d like everything planned and priced in EUR'
        },
        'travel': {
            name: '✈️ Travel Specialist',
            description: 'Transportation & Flight Expert',
            capabilities: 'I specialize in finding the best transportation options:<ul style="margin-top: 10px; line-height: 1.8;"><li>✈️ Flight bookings with real-time pricing</li><li>🚂 Train schedules and routes</li><li>🚌 Bus and ground transportation</li><li>💰 Price comparisons across providers</li><li>⏱️ Journey duration optimization</li></ul>',
            example: '✈️ Find me the best flights from New York (JFK) to Tokyo for a 10-day trip in March. Budget is $1,500 USD for flights'
        },
        'accommodation': {
            name: '🏨 Accommodation Specialist',
            description: 'Hotel & Lodging Expert',
            capabilities: 'I find the perfect place for your stay:<ul style="margin-top: 10px; line-height: 1.8;"><li>🏨 Hotel recommendations with ratings</li><li>💰 Price per night and total costs</li><li>⭐ Reviews and amenities analysis</li><li>📍 Location-based suggestions</li><li>🛏️ Budget-conscious options</li></ul>',
            example: '🏨 Find hotels in Mumbai for 6 nights with a budget of $2,400 USD total. I prefer 4-star hotels near tourist attractions'
        },
        'itinerary': {
            name: '🗓️ Itinerary Designer',
            description: 'Activity Planning Expert',
            capabilities: 'I create personalized day-by-day plans:<ul style="margin-top: 10px; line-height: 1.8;"><li>📅 Optimized daily schedules</li><li>🎯 Activities matching your travel style</li><li>💰 Cost estimates per activity</li><li>🍽️ Meal recommendations</li><li>🚇 Local transport suggestions</li><li>💡 Insider tips and best times to visit</li></ul>',
            example: '🗓️ Create a 5-day itinerary for Tokyo with a mix of sightseeing and relaxation. Budget around $800 USD for activities'
        }
    };
    
    function updateWelcomeMessage() {
        const selectedAgent = getSelectedAgent();
        const info = agentInfo[selectedAgent];
        
        const welcomeSenderName = document.getElementById('welcomeSenderName');
        const welcomeContent = document.getElementById('welcomeContent');
        
        welcomeSenderName.textContent = info.name;
        welcomeContent.innerHTML = `
            <h2>Welcome to ${info.description}!</h2>
            <p>${info.capabilities}</p>
            <p style="margin-top: 15px; padding: 15px; background: rgba(0,123,255,0.1); border-radius: 8px; border-left: 4px solid #007bff;">
                <strong>💡 Ready to help!</strong> Type your query below or click the example to get started.
            </p>
        `;
        
        updateExampleQuery();
    }
    
    function updateExampleQuery() {
        const selectedAgent = getSelectedAgent();
        const info = agentInfo[selectedAgent];
        const exampleContainer = document.getElementById('exampleContainer');
        
        exampleContainer.innerHTML = `
            <button class="example-btn" onclick="setExample(this)">${info.example}</button>
        `;
    }
    
    function toggleTheme() {
        const html = document.documentElement;
        const currentTheme = html.getAttribute('data-theme') || 'light';
        const newTheme = currentTheme === 'light' ? 'dark' : 'light';
        
        html.setAttribute('data-theme', newTheme);
        localStorage.setItem('theme', newTheme);
        
        const icon = document.getElementById('themeIcon');
        const text = document.getElementById('themeText');
        
        if (newTheme === 'dark') {
            icon.textContent = '☀️';
            text.textContent = 'Light';
        } else {
            icon.textContent = '🌙';
            text.textContent = 'Dark';
        }
    }
    
    document.addEventListener('DOMContentLoaded', function() {
        const savedTheme = localStorage.getItem('theme') || 'light';
        document.documentElement.setAttribute('data-theme', savedTheme);
        
        const icon = document.getElementById('themeIcon');
        const text = document.getElementById('themeText');
        
        if (savedTheme === 'dark') {
            icon.textContent = '☀️';
            text.textContent = 'Light';
        }
        
        document.getElementById('messageInput').focus();
        updateExampleQuery();
    });
    
    let isProcessing = false;
    
    function setExample(button) {
        if (!isProcessing) {
            const text = button.textContent || button.innerText;
            document.getElementById('messageInput').value = text;
            document.getElementById('messageInput').focus();
        }
    }
    
    function handleKeyPress(event) {
        if (event.key === 'Enter' && !event.shiftKey) {
            event.preventDefault();
            sendMessage();
        }
    }
    
    function getSelectedAgent() {
        const radios = document.getElementsByName('agent');
        for (let radio of radios) {
            if (radio.checked) {
                return radio.value;
            }
        }
        return 'orchestrator';
    }
    
    async function sendMessage() {
        if (isProcessing) return;
        const input = document.getElementById('messageInput');
        const message = input.value.trim();
        if (!message) return;
        
        const selectedAgent = getSelectedAgent();
        
        addMessage('user', message);
        input.value = '';
        isProcessing = true;
        document.getElementById('sendBtn').disabled = true;
        document.getElementById('sendBtn').innerHTML = '<div class="loading"></div>';
        
        const typingIndicator = document.createElement('div');
        typingIndicator.className = 'typing-indicator';
        typingIndicator.id = 'typingIndicator';
        
        const agentNames = {
            'orchestrator': 'Full Travel Orchestrator',
            'travel': 'Travel Specialist',
            'accommodation': 'Accommodation Specialist',
            'itinerary': 'Itinerary Designer'
        };
        typingIndicator.innerHTML = `✨ ${agentNames[selectedAgent]} is working on your request...`;
        
        document.getElementById('chatMessages').appendChild(typingIndicator);
        document.getElementById('chatMessages').scrollTop = document.getElementById('chatMessages').scrollHeight;
        
        try {
            const response = await fetch('/chat', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ 
                    message: message,
                    agent: selectedAgent 
                })
            });
            const data = await response.json();
            const indicator = document.getElementById('typingIndicator');
            if (indicator) indicator.remove();
            addMessage('agent', data.response || '❌ Unexpected response.');
        } catch (error) {
            const indicator = document.getElementById('typingIndicator');
            if (indicator) indicator.remove();
            addMessage('agent', '❌ Sorry, I encountered an error. Please try again.');
            console.error('Error:', error);
        } finally {
            isProcessing = false;
            document.getElementById('sendBtn').disabled = false;
            document.getElementById('sendBtn').innerHTML = 'Send';
        }
    }
    
    function addMessage(sender, content) {
        const chatMessages = document.getElementById('chatMessages');
        const messageDiv = document.createElement('div');
        messageDiv.className = `message ${sender}-message`;
        const messageBubble = document.createElement('div');
        messageBubble.className = 'message-bubble';
        const senderName = document.createElement('div');
        senderName.className = 'sender-name';
        
        const selectedAgent = getSelectedAgent();
        senderName.innerHTML = sender === 'user' ? '👤 You' : agentInfo[selectedAgent].name;
        
        const messageContent = document.createElement('div');
        if (sender === 'agent') {
            messageContent.className = 'card-content';
            messageContent.innerHTML = content;
        } else {
            messageContent.textContent = content;
        }
        messageBubble.appendChild(senderName);
        messageBubble.appendChild(messageContent);
        messageDiv.appendChild(messageBubble);
        chatMessages.appendChild(messageDiv);
        chatMessages.scrollTop = chatMessages.scrollHeight;
    }
</script>
</body>
</html>
'''

In [ ]:
# ============= HELPER FUNCTIONS FOR RESPONSE PROCESSING =============

def extract_response_from_events(events):
    """Extract response from events with proper error detection"""
    if not events:
        return None, "Sorry, I couldn't generate a response. Please try again."
    
    for event in events:
        if hasattr(event, 'author') and event.author == 'AggregatorAgent':
            if hasattr(event, 'actions') and event.actions:
                if hasattr(event.actions, 'state_delta') and 'final_trip_summary' in event.actions.state_delta:
                    summary_json = event.actions.state_delta['final_trip_summary']
                    
                    if isinstance(summary_json, dict):
                        validation_status = summary_json.get('validation_status', 'OK')
                        if validation_status == "ERROR":
                            return "ERROR", summary_json
                        else:
                            return "SUCCESS", summary_json
                    
                    return "SUCCESS", summary_json
    
    return "UNKNOWN", "Unable to process your request. Please try again."


def format_error_response(error_data):
    """Format validation error response from JSON data"""
    if isinstance(error_data, str):
        try:
            error_json = json.loads(error_data)
        except:
            return f'<div class="travel-card error-section"><div class="card-content"><p>{html.escape(error_data)}</p></div></div>'
    else:
        error_json = error_data
    
    invalid_params = error_json.get('invalid_parameters', [])
    human_summary = error_json.get('human_summary', 'Please provide more details.')
    
    if not invalid_params:
        return f'''
        <div class="travel-card error-section">
            <div class="card-header">
                <span class="card-icon">⚠️</span>
                <h2 class="card-title">Validation Error</h2>
            </div>
            <div class="card-content">
                <p>{html.escape(human_summary)}</p>
            </div>
        </div>
        '''
    
    params_html = ''.join([
        f'<li><strong>{html.escape(param.get("name", ""))}</strong>: {html.escape(param.get("reason", ""))}</li>' 
        for param in invalid_params
    ])
    
    return f'''
    <div class="travel-card incomplete-section">
        <div class="card-header">
            <span class="card-icon">📝</span>
            <h2 class="card-title">Additional Information Needed</h2>
        </div>
        <div class="card-content">
            <p><strong>{html.escape(human_summary)}</strong></p>
            <ul class="feature-list" style="margin-top: 15px;">
                {params_html}
            </ul>
            <p style="margin-top: 20px; font-style: italic; color: #666;">
                💡 <strong>Tip:</strong> Please provide all the missing information in your next message, 
                and I'll create a comprehensive travel plan for you!
            </p>
        </div>
    </div>
    '''


def format_success_response(success_data):
    """Format successful JSON response into structured card-based HTML with color-coded borders and PRESERVED URLs"""
    if isinstance(success_data, str):
        try:
            data = json.loads(success_data)
        except:
            data = {}
    else:
        data = success_data
    
    cards_html = '<div class="travel-plan-container">'
    
    # 1. Trip Overview Card - BLUE
    if data.get('overall_summary'):
        cards_html += f'''
        <div class="travel-card overview">
            <div class="card-header">
                <span class="card-icon">🌍</span>
                <h2 class="card-title">Trip Overview</h2>
            </div>
            <div class="card-content">
                <p>{html.escape(data['overall_summary'])}</p>
                <div style="margin-top: 15px; padding: 15px; background: rgba(0,123,255,0.08); border-radius: 8px;">
                    <p><strong>📍 Route:</strong> {html.escape(data.get('start_location', 'N/A'))} → {html.escape(data.get('end_location', 'N/A'))}</p>
                    <p><strong>📅 Duration:</strong> {data.get('trip_duration_days', 'N/A')} days</p>
                    <p><strong>💰 Budget:</strong> {data.get('preferred_currency', 'USD')} {data.get('total_budget', 0):,.2f}</p>
                    <p><strong>🎯 Style:</strong> {html.escape(data.get('travel_style', 'N/A'))}</p>
                    <p><strong>🍽️ Dietary:</strong> {html.escape(data.get('dietary_preference', 'N/A'))}</p>
                </div>
            </div>
        </div>
        '''
    
    # 2. Transport Options Card - GREEN
    transport_options = data.get('transport_options', [])
    transport_summary = data.get('transport_summary_text', '')
    
    if transport_summary or transport_options:
        cards_html += f'''
        <div class="travel-card transport">
            <div class="card-header">
                <span class="card-icon">✈️</span>
                <h2 class="card-title">Transportation Options</h2>
            </div>
            <div class="card-content">
        '''
        
        if transport_summary:
            cards_html += f'<p style="margin-bottom: 20px;">{html.escape(transport_summary)}</p>'
        
        for idx, route in enumerate(transport_options, 1):
            currency = html.escape(route.get('currency', 'USD'))
            price = route.get('total_price', 0)
            
            cards_html += f'''
            <div class="option-card">
                <h4>{html.escape(route.get('label', 'Route'))}</h4>
                <p><strong>💰 Total Price:</strong> {currency} {price:,.2f}</p>
                <p><strong>⏱️ Duration:</strong> {html.escape(route.get('total_duration', 'N/A'))}</p>
                <p><strong>🚗 Mode:</strong> {html.escape(route.get('main_mode', 'N/A'))}</p>
            '''
            
            if route.get('pros'):
                cards_html += f'<p><strong>✅ Pros:</strong> {html.escape(route["pros"])}</p>'
            if route.get('cons'):
                cards_html += f'<p><strong>⚠️ Cons:</strong> {html.escape(route["cons"])}</p>'
            
            legs = route.get('legs', [])
            if legs:
                cards_html += '<p><strong>Journey Details:</strong></p><ul>'
                for leg in legs:
                    leg_price = leg.get('price', 0)
                    booking_url = leg.get('booking_url', '')
                    
                    cards_html += f'''
                    <li>{html.escape(leg.get('mode', 'N/A').title())}: 
                        {html.escape(leg.get('from_location', 'N/A'))} → {html.escape(leg.get('to_location', 'N/A'))} 
                        ({html.escape(leg.get('duration', 'N/A'))}) - {currency} {leg_price:,.2f}
                    '''
                    
                    if booking_url:
                        cards_html += f' | <a href="{html.escape(booking_url)}" target="_blank" rel="noopener">🔗 Book Now</a>'
                    
                    cards_html += '</li>'
                cards_html += '</ul>'
            
            cards_html += '</div>'
        
        cards_html += '</div></div>'
    
    # 3. Accommodation Card - RED
    primary_hotel = data.get('primary_accommodation')
    accommodation_summary = data.get('accommodation_summary_text', '')
    
    if accommodation_summary or primary_hotel:
        cards_html += f'''
        <div class="travel-card accommodation">
            <div class="card-header">
                <span class="card-icon">🏨</span>
                <h2 class="card-title">Accommodation</h2>
            </div>
            <div class="card-content">
        '''
        
        if accommodation_summary:
            cards_html += f'<p style="margin-bottom: 20px;">{html.escape(accommodation_summary)}</p>'
        
        if primary_hotel:
            currency = html.escape(primary_hotel.get('currency', 'USD'))
            per_night = primary_hotel.get('price_per_night', 0)
            total = primary_hotel.get('total_price', 0)
            rating = primary_hotel.get('rating', 0)
            hotel_url = primary_hotel.get('url', '')
            
            cards_html += f'''
            <div class="option-card">
                <h4>{html.escape(primary_hotel.get('name', 'Hotel'))}</h4>
                <p><strong>📍 Location:</strong> {html.escape(primary_hotel.get('address', 'N/A'))}</p>
                <p><strong>💰 Price:</strong> {currency} {per_night:,.2f}/night (Total: {currency} {total:,.2f})</p>
                <p><strong>⭐ Rating:</strong> {rating}/5.0 ({primary_hotel.get('reviews_count', 0)} reviews)</p>
                <p><strong>🔖 Provider:</strong> {html.escape(primary_hotel.get('provider', 'N/A'))}</p>
            '''
            
            if hotel_url:
                cards_html += f'<p><a href="{html.escape(hotel_url)}" target="_blank" rel="noopener">🔗 View Hotel Details & Book</a></p>'
            
            cards_html += '</div>'
        
        cards_html += '</div></div>'
    
    # 4. Itinerary Card - YELLOW
    itinerary_days = data.get('itinerary_days', [])
    itinerary_summary = data.get('itinerary_summary_text', '')
    
    if itinerary_summary or itinerary_days:
        cards_html += f'''
        <div class="travel-card itinerary">
            <div class="card-header">
                <span class="card-icon">🗓️</span>
                <h2 class="card-title">Day-by-Day Itinerary</h2>
            </div>
            <div class="card-content">
        '''
        
        if itinerary_summary:
            cards_html += f'<p style="margin-bottom: 20px;">{html.escape(itinerary_summary)}</p>'
        
        for day in itinerary_days:
            day_num = day.get('day_number', 0)
            cards_html += f'''
            <div class="option-card">
                <h4>Day {day_num}: {html.escape(day.get('title', 'Activities'))}</h4>
                <p style="margin-bottom: 15px;">{html.escape(day.get('overview', ''))}</p>
            '''
            
            activities = day.get('activities', [])
            if activities:
                cards_html += '<p><strong>📍 Activities:</strong></p><ul>'
                for activity in activities:
                    time_of_day = activity.get('time_of_day', 'day').title()
                    activity_cost = activity.get('cost', 0)
                    currency = activity.get('currency', 'USD')
                    booking_url = activity.get('booking_url', '')
                    
                    cards_html += f'''
                    <li><strong>{time_of_day}:</strong> {html.escape(activity.get('title', 'Activity'))} 
                        at {html.escape(activity.get('location', 'N/A'))} 
                        ({html.escape(activity.get('duration', 'N/A'))}) - {currency} {activity_cost:,.2f}
                    '''
                    
                    if booking_url:
                        cards_html += f' | <a href="{html.escape(booking_url)}" target="_blank" rel="noopener">🔗 Book</a>'
                    
                    if activity.get('notes'):
                        cards_html += f'<br><em>{html.escape(activity["notes"])}</em>'
                    
                    cards_html += '</li>'
                cards_html += '</ul>'
            
            meals = day.get('meals', [])
            if meals:
                cards_html += '<p><strong>🍽️ Meals:</strong></p><ul>'
                for meal in meals:
                    meal_type = meal.get('meal_type', 'meal').title()
                    meal_url = meal.get('booking_url', '')
                    
                    cards_html += f'''
                    <li><strong>{meal_type}:</strong> {html.escape(meal.get('place_name', 'Restaurant'))} 
                        - {html.escape(meal.get('cuisine', 'N/A'))} cuisine 
                        ({html.escape(meal.get('price_range', 'N/A'))})
                    '''
                    
                    if meal_url:
                        cards_html += f' | <a href="{html.escape(meal_url)}" target="_blank" rel="noopener">🔗 View Menu</a>'
                    
                    if meal.get('notes'):
                        cards_html += f'<br><em>{html.escape(meal["notes"])}</em>'
                    
                    cards_html += '</li>'
                cards_html += '</ul>'
            
            if day.get('local_transport_notes'):
                cards_html += f'<p><strong>🚇 Transport:</strong> {html.escape(day["local_transport_notes"])}</p>'
            
            cards_html += '</div>'
        
        cards_html += '</div></div>'
    
    # 5. Food Recommendations Card - PURPLE
    food_highlights = data.get('food_highlights', [])
    food_summary = data.get('food_summary_text', '')
    
    if food_summary or food_highlights:
        cards_html += f'''
        <div class="travel-card food">
            <div class="card-header">
                <span class="card-icon">🍽️</span>
                <h2 class="card-title">Food Recommendations</h2>
            </div>
            <div class="card-content">
        '''
        
        if food_summary:
            cards_html += f'<p style="margin-bottom: 20px;">{html.escape(food_summary)}</p>'
        
        for restaurant in food_highlights:
            rating = restaurant.get('rating', 0)
            restaurant_url = restaurant.get('url', '')
            
            cards_html += f'''
            <div class="option-card">
                <h4>{html.escape(restaurant.get('name', 'Restaurant'))}</h4>
                <p><strong>📍 Location:</strong> {html.escape(restaurant.get('address', 'N/A'))}</p>
                <p><strong>🍴 Cuisine:</strong> {html.escape(restaurant.get('cuisine', 'N/A'))}</p>
                <p><strong>💰 Price:</strong> {html.escape(restaurant.get('price_level', 'N/A'))}</p>
                <p><strong>⭐ Rating:</strong> {rating}/5.0 ({restaurant.get('reviews_count', 0)} reviews)</p>
                <p><strong>🥗 Dietary Fit:</strong> {html.escape(restaurant.get('dietary_fit', 'N/A'))}</p>
            '''
            
            if restaurant_url:
                cards_html += f'<p><a href="{html.escape(restaurant_url)}" target="_blank" rel="noopener">🔗 View on Maps & Reviews</a></p>'
            
            dishes = restaurant.get('must_try_dishes', [])
            if dishes:
                cards_html += '<p><strong>🌟 Must-Try Dishes:</strong></p><ul>'
                for dish in dishes:
                    dish_price = dish.get('typical_price', 0)
                    currency = dish.get('currency', 'USD')
                    cards_html += f'''
                    <li>{html.escape(dish.get('name', 'Dish'))} - {currency} {dish_price:,.2f}
                        <br><em>{html.escape(dish.get('description', ''))}</em>
                    </li>
                    '''
                cards_html += '</ul>'
            
            if restaurant.get('notes'):
                cards_html += f'<p><strong>💡 Note:</strong> {html.escape(restaurant["notes"])}</p>'
            
            cards_html += '</div>'
        
        cards_html += '</div></div>'
    
    # 6. Missing Components Warning
    missing = data.get('missing_components', [])
    if missing:
        cards_html += f'''
        <div class="travel-card" style="border-left-color: #ffc107;">
            <div class="card-header">
                <span class="card-icon">⚠️</span>
                <h2 class="card-title">Incomplete Data</h2>
            </div>
            <div class="card-content">
                <p>Some components could not be fully processed:</p>
                <ul>
        '''
        for component in missing:
            cards_html += f'<li>{html.escape(component)}</li>'
        cards_html += '</ul></div></div>'
    
    cards_html += '</div>'
    return cards_html


def format_simple_response(response_text):
    """Format standalone agent response using markdown to HTML - PRESERVE URLs"""
    try:
        html_content = markdown.markdown(
            str(response_text),
            extensions=['extra', 'nl2br', 'sane_lists']
        )
        
        # Keep URLs but make them safe
        allowed_tags = [
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6',
            'p', 'br', 'strong', 'em', 'u', 'del',
            'ul', 'ol', 'li',
            'code', 'pre', 'blockquote',
            'table', 'thead', 'tbody', 'tr', 'th', 'td',
            'hr', 'span', 'div', 'a'  # Added 'a' for links
        ]
        
        allowed_attributes = {
            'code': ['class'],
            'span': ['class'],
            'div': ['class'],
            'a': ['href', 'target', 'rel']  # Allow link attributes
        }
        
        clean_html = bleach.clean(
            html_content,
            tags=allowed_tags,
            attributes=allowed_attributes,
            strip=True
        )
        
        # Ensure all links open in new tab with security
        clean_html = re.sub(
            r'<a\s+href="([^"]+)"',
            r'<a href="\1" target="_blank" rel="noopener noreferrer"',
            clean_html
        )
        
        return f'''
        <div class="travel-card">
            <div class="card-content">
                {clean_html}
            </div>
        </div>
        '''
    except Exception as e:
        print(f"❌ Error formatting markdown: {e}")
        # Fallback: preserve URLs in plain text conversion
        response_html = html.escape(str(response_text))
        
        # Convert URLs to clickable links
        url_pattern = r'(https?://[^\s<>"]+|www\.[^\s<>"]+)'
        response_html = re.sub(
            url_pattern,
            r'<a href="\1" target="_blank" rel="noopener noreferrer">\1</a>',
            response_html
        )
        
        response_html = response_html.replace('\n\n', '</p><p>')
        response_html = response_html.replace('\n', '<br>')
        response_html = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', response_html)
        
        return f'''
        <div class="travel-card">
            <div class="card-content">
                <p>{response_html}</p>
            </div>
        </div>
        '''

In [ ]:
# ============= FLASK ROUTES =============

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        user_message = data.get('message', '').strip()
        selected_agent = data.get('agent', 'orchestrator')
        
        if not user_message:
            return jsonify({'error': 'Empty message'}), 400

        print(f"📨 Received query for {selected_agent}: {user_message}")

        def run_async():
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            try:
                if selected_agent == 'travel':
                    return loop.run_until_complete(process_standalone_agent(travel_standalone_system, user_message))
                elif selected_agent == 'accommodation':
                    return loop.run_until_complete(process_standalone_agent(accommodation_standalone_system, user_message))
                elif selected_agent == 'itinerary':
                    return loop.run_until_complete(process_standalone_agent(itinerary_standalone_system, user_message))
                else:  # orchestrator
                    return loop.run_until_complete(process_travel_query(user_message))
            finally:
                loop.close()

        status, response_data = run_async()

        if selected_agent == 'orchestrator':
            if status == "ERROR":
                html_response = format_error_response(response_data)
            elif status == "SUCCESS":
                html_response = format_success_response(response_data)
            else:
                html_response = format_simple_response(response_data)
        else:
            html_response = format_simple_response(response_data)

        return jsonify({'response': html_response})

    except Exception as e:
        print(f"❌ Error in chat endpoint: {str(e)}")
        import traceback
        traceback.print_exc()
        
        error_html = f'''
        <div class="travel-card error-section">
            <div class="card-header">
                <span class="card-icon">❌</span>
                <h2 class="card-title">System Error</h2>
            </div>
            <div class="card-content">
                <p>Sorry, something went wrong while processing your request.</p>
                <p style="margin-top: 10px; font-size: 0.9em; color: #666;">
                    <em>Error details: {html.escape(str(e))}</em>
                </p>
                <p style="margin-top: 15px;">Please try again or rephrase your query.</p>
            </div>
        </div>
        '''
        return jsonify({'response': error_html}), 500


async def process_standalone_agent(agent_system, query):
    """Process query using standalone agent"""
    try:
        print(f"🔄 Processing with standalone agent: {query}")
        response_text = await agent_system.process_query(query)
        print(f"✅ Standalone agent response received")
        return "SUCCESS", response_text
    except Exception as e:
        error_msg = f"Error processing query: {str(e)}"
        print(f"❌ {error_msg}")
        return "ERROR", error_msg


async def process_travel_query(query):
    """Process travel query using the existing travel system"""
    try:
        print(f"🔄 Processing: {query}")
        events = await travel_system.process_query(query)
        status, response = extract_response_from_events(events)
        print(f"✅ Query processed with status: {status}")
        return status, response
    except Exception as e:
        error_msg = f"Error processing query: {str(e)}"
        print(f"❌ {error_msg}")
        return "ERROR", error_msg


@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

In [ ]:
def start_web_server(port=8050, host='0.0.0.0'):
    """Start the Flask web server"""
    print(f"\nStarting TravelConcierge AI Web Interface...")
    print(f"Local URL: http://localhost:{port}")
    print(f"Network URL: http://{host}:{port}")
    print("\nTo stop the server: Press Ctrl+C")
    print("-" * 60)
    app.run(host=host, port=port, debug=False, use_reloader=False)


print("Complete web interface ready!")
print("Run: start_web_server()")

## Launch Web Interface

Run the cell below to start the professional web interface on your local machine.

In [ ]:
# Uncomment the line below to start the web interface:
start_web_server()